In [1]:
%load_ext autoreload
%autoreload 2
from hmpai.pytorch.generators import SAT1DataLoader
import xarray as xr
from pathlib import Path
import torch
from hmpai.pytorch.models import *
from hmpai.training import split_data_on_participants
from hmpai.pytorch.training import train, validate, calculate_class_weights, train_and_test
from hmpai.pytorch.utilities import DEVICE
from hmpai.normalization import *
import random
import numpy as np
from torchinfo import summary
from tqdm.notebook import tqdm, trange

In [3]:
random.seed(42)
np.random.seed(42)
data_path = Path("../data/sat1/split_stage_data.nc")

dataset = xr.load_dataset(data_path)
train_data, val_data, test_data = split_data_on_participants(dataset, 60, norm_0_to_1)

# eeg_dataset = EegDataset(data)
# train_loader = SAT1DataLoader(train_data)
# val_loader = SAT1DataLoader(val_data)
# test_loader = SAT1DataLoader(test_data)

In [5]:
model = SAT1Base(len(dataset.channels), len(dataset.samples), len(dataset.labels)).to(
    DEVICE
)

In [ ]:
summary(model, (16, 1, 147, 30))

In [6]:
train_and_test(model, train_data, test_data, val_data, logs_path=Path("../logs/"))

  0%|          | 0/757 [00:00<?, ? batch/s]

  0%|          | 0/757 [00:00<?, ? batch/s]

  0%|          | 0/757 [00:00<?, ? batch/s]

  0%|          | 0/757 [00:00<?, ? batch/s]

  0%|          | 0/757 [00:00<?, ? batch/s]

...

  0%|          | 0/757 [00:00<?, ? batch/s]

  0%|          | 0/757 [00:00<?, ? batch/s]

  0%|          | 0/757 [00:00<?, ? batch/s]

  0%|          | 0/757 [00:00<?, ? batch/s]

{'confirmation': {'precision': 0.851528384279476,
  'recall': 0.8496732026143791,
  'f1-score': 0.8505997818974919,
  'support': 459.0},
 'decision': {'precision': 0.8893178893178894,
  'recall': 0.7755331088664422,
  'f1-score': 0.8285371702637889,
  'support': 891.0},
 'encoding': {'precision': 0.7869615832363213,
  'recall': 0.7586980920314254,
  'f1-score': 0.7725714285714285,
  'support': 891.0},
 'pre-attentive': {'precision': 0.6806122448979591,
  'recall': 0.7828638497652582,
  'f1-score': 0.7281659388646289,
  'support': 852.0},
 'response': {'precision': 0.8615384615384616,
  'recall': 0.8799102132435466,
  'f1-score': 0.8706274292059967,
  'support': 891.0},
 'accuracy': 0.8052208835341366,
 'macro avg': {'precision': 0.8139917126540215,
  'recall': 0.8093356933042102,
  'f1-score': 0.8101003497606669,
  'support': 3984.0},
 'weighted avg': {'precision': 0.8112271436530902,
  'recall': 0.8052208835341366,
  'f1-score': 0.8065109138524935,
  'support': 3984.0}}

In [20]:
# progress = trange(epochs)
for epoch in range(epochs):
    # progress.set_description(f"Epoch {epoch}")
    with tqdm(total=len(train_loader), unit=" batch") as tepoch:
        tepoch.set_description(f"Epoch {epoch + 1}")
        batch_losses = train(model, train_loader, opt, loss, tepoch)
        # Shuffle data before next epoch
        train_loader.shuffle()

        val_losses, val_accuracy = validate(model, val_loader, loss)
        tepoch.set_postfix(
            {
                "loss": np.mean(batch_losses),
                "val_loss": np.mean(val_losses),
                "val_accuracy": val_accuracy,
            }
        )
        # tepoch.set_postfix_str(
        #     f"epoch {epoch}, loss: {np.mean(batch_losses)}, val_loss: {np.mean(val_losses)}, val_accuracy: {val_accuracy}"
        # )

  0%|          | 0/757 [00:00<?, ? batch/s]

  0%|          | 0/757 [00:00<?, ? batch/s]

  0%|          | 0/757 [00:00<?, ? batch/s]

  0%|          | 0/757 [00:00<?, ? batch/s]

KeyboardInterrupt: 